<a href="https://colab.research.google.com/github/gunjanofficial/Sentiment_Analysis_NLP_Project/blob/main/SentimentAnalysis_project_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
# Installing the kaggle labraries

In [29]:
!pip install kaggle


In [ ]:
# Configration the path of the kaggle.json

In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
#Imoprting the Twitter sentiments

In [ ]:
# API to fetch the dataset from kaggle

In [5]:
!kaggle datasets download -d kazanova/sentiment140      # shift + enter

 90% 73.0M/80.9M [00:00<00:00, 104MB/s]
100% 80.9M/80.9M [00:00<00:00, 99.9MB/s]


In [6]:
#extracting the compressed dataset
from zipfile import ZipFile
dataset='/content/sentiment140.zip'

with ZipFile(dataset,'r') as zip:
  zip.extractall()
  print('This dataset is extracted')


This dataset is extracted


Importing the Dependencies
1.import re: The re module in Python is used for working with regular expressions. Regular expressions are powerful tools for pattern matching and text manipulation. With regular expressions, you can search for patterns within strings, extract specific information, replace text, and more.
2.nltk.corpus.stopwords:Stop words are words that are commonly used in natural language but typically do not contribute much to the meaning of a sentence, such as "the", "is", "and", etc. These words are often removed from text data during preprocessing to improve the performance of NLP tasks like text classification, sentiment analysis, or information retrieval.
3.nltk.stem.porter.PorterStemmer: This import gives access to the Porter Stemmer algorithm implemented in NLTK. Stemming is the process of reducing words to their root or base form, often by removing suffixes. The Porter Stemmer is a widely-used algorithm for English stemming that applies a set of rules to reduce words to their stem. For example, "running" and "runs" would both be stemmed to "run".
4.sklearn.feature_extraction.text.TfidfVectorizer: This import provides access to the TF-IDF vectorizer class from scikit-learn (sklearn). TF-IDF stands for Term Frequency-Inverse Document Frequency. It is a technique used to convert textual data into numerical vectors, where each feature represents a word or phrase.TF-IDF takes into account both the frequency of a term in a document (term frequency) and its rarity across documents (inverse document frequency), providing a measure of how important a word is to a document within a corpus.

In [7]:
import numpy as np
import pandas as pd
import re           # patter matching and text manipulations
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [8]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [9]:
#prining the stop words in english
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [10]:
# Loading the data from csv file to pandas datadaframe
twitter_data=pd.read_csv('/content/training.1600000.processed.noemoticon.csv',encoding='ISO-8859-1')

In [11]:
# checking the number of row and columns
twitter_data.shape

(1599999, 6)

In [12]:
# printing the frist 5 row
twitter_data.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [13]:
twitter_data.columns

Index(['0', '1467810369', 'Mon Apr 06 22:19:45 PDT 2009', 'NO_QUERY',
       '_TheSpecialOne_',
       '@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer.  You shoulda got David Carr of Third Day to do it. ;D'],
      dtype='object')

In [14]:
# naming the columns and reading the dataset again
columns_names=['Target','id','date','flag','user','text']
# Loading the data from csv file to pandas datadaframe
twitter_data=pd.read_csv('/content/training.1600000.processed.noemoticon.csv',names=columns_names, encoding='ISO-8859-1')

In [15]:
# checked the rows and columns
twitter_data.shape

(1600000, 6)

In [16]:
# checked the first 5rows
twitter_data.head()

,Target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [17]:
# cheked the last 5 rows
twitter_data.tail()

,Target,id,date,flag,user,text
1599995,4,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599996,4,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599997,4,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599998,4,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...
1599999,4,2193602129,Tue Jun 16 08:40:50 PDT 2009,NO_QUERY,RyanTrevMorris,happy #charitytuesday @theNSPCC @SparksCharity...


In [18]:
# checked the null values in dataset
twitter_data.isnull().sum()

Target    0
id        0
date      0
flag      0
user      0
text      0
dtype: int64

In [19]:
# checking the distribution of target columns
twitter_data['Target'].value_counts()    # that means the dataset equally distributed

0    800000
4    800000
Name: Target, dtype: int64

convert the target "4" to "1" that is Positive



In [20]:

twitter_data.replace({'Target':{4:1}},inplace=True)
twitter_data

,Target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
...,...,...,...,...,...,...
1599995,1,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...
1599996,1,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...
1599997,1,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...
1599998,1,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...


In [21]:
# checking the distribution of target columns
twitter_data['Target'].value_counts()    # that means the dataset equally distributed

0    800000
1    800000
Name: Target, dtype: int64

0--> Negative Tweet

1--> Positive Tweet

## stemming ##

stemming is the process of reducing a word to its root word
example: actor,actress,acting=act,running=run.

In [22]:
port_stem=PorterStemmer()

In [23]:
def stemming (content):

  stemmed_content=re.sub('[^a-zA-Z]',' ',content)    # ^ means removing all the punctuation and text which is not belong to "a" to "z" or "A" to "Z".
  stemmed_content=stemmed_content.lower()            # convert all the content in lower case
  stemmed_content=stemmed_content.split()             # split the string into a list of words
  stemmed_content=[port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]   # if the word not in stopword we will do the port_stem process.
  stemmed_content=" ".join(stemmed_content)

  return stemmed_content

In [24]:
# joins the list of stemmed words back into a single string with words separated by spaces.so befor stemmed words are["run",'walk',"paly"] and afte execute this line stemmed words like ' run walk play '
twitter_data['stemmed_content']=twitter_data['text'].apply(stemming)   #50minutes to completr for execution


In [25]:
print(twitter_data[['Target','stemmed_content']])

         Target                                    stemmed_content
0             0  switchfoot http twitpic com zl awww bummer sho...
1             0  upset updat facebook text might cri result sch...
2             0  kenichan dive mani time ball manag save rest g...
3             0                    whole bodi feel itchi like fire
4             0                      nationwideclass behav mad see
...         ...                                                ...
1599995       1                         woke school best feel ever
1599996       1  thewdb com cool hear old walt interview http b...
1599997       1                       readi mojo makeov ask detail
1599998       1  happi th birthday boo alll time tupac amaru sh...
1599999       1  happi charitytuesday thenspcc sparkschar speak...

[1600000 rows x 2 columns]


In [26]:
# Separating the data and label
x=twitter_data['stemmed_content'].values
y=twitter_data['Target'].values

In [27]:
print(x)

['switchfoot http twitpic com zl awww bummer shoulda got david carr third day'
 'upset updat facebook text might cri result school today also blah'
 'kenichan dive mani time ball manag save rest go bound' ...
 'readi mojo makeov ask detail'
 'happi th birthday boo alll time tupac amaru shakur'
 'happi charitytuesday thenspcc sparkschar speakinguph h']


In [28]:
print(y)

[0 0 0 ... 1 1 1]


Splitting the data to training and test data

In [29]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,stratify=y,random_state=2)

In [30]:
print(x.shape,x_train.shape,x_test.shape)

(1600000,) (1280000,) (320000,)


In [31]:
print(y.shape,y_train.shape,y_test.shape)

(1600000,) (1280000,) (320000,)


In [32]:
# converting the textual data to numerical data

vectorizer=TfidfVectorizer()

x_train=vectorizer.fit_transform(x_train)
x_test=vectorizer.transform(x_test)

In [33]:
print(x_train)

  (0, 443066)	0.4484755317023172
  (0, 235045)	0.41996827700291095
  (0, 109306)	0.3753708587402299
  (0, 185193)	0.5277679060576009
  (0, 354543)	0.3588091611460021
  (0, 436713)	0.27259876264838384
  (1, 160636)	1.0
  (2, 288470)	0.16786949597862733
  (2, 132311)	0.2028971570399794
  (2, 150715)	0.18803850583207948
  (2, 178061)	0.1619010109445149
  (2, 409143)	0.15169282335109835
  (2, 266729)	0.24123230668976975
  (2, 443430)	0.3348599670252845
  (2, 77929)	0.31284080750346344
  (2, 433560)	0.3296595898028565
  (2, 406399)	0.32105459490875526
  (2, 129411)	0.29074192727957143
  (2, 407301)	0.18709338684973031
  (2, 124484)	0.1892155960801415
  (2, 109306)	0.4591176413728317
  (3, 172421)	0.37464146922154384
  (3, 411528)	0.27089772444087873
  (3, 388626)	0.3940776331458846
  (3, 56476)	0.5200465453608686
  :	:
  (1279996, 390130)	0.22064742191076112
  (1279996, 434014)	0.2718945052332447
  (1279996, 318303)	0.21254698865277746
  (1279996, 237899)	0.2236567560099234
  (1279996, 2910

In [34]:
print(x_test)

  (0, 420984)	0.17915624523539803
  (0, 409143)	0.31430470598079707
  (0, 398906)	0.3491043873264267
  (0, 388348)	0.21985076072061738
  (0, 279082)	0.1782518010910344
  (0, 271016)	0.4535662391658828
  (0, 171378)	0.2805816206356073
  (0, 138164)	0.23688292264071403
  (0, 132364)	0.25525488955578596
  (0, 106069)	0.3655545001090455
  (0, 67828)	0.26800375270827315
  (0, 31168)	0.16247724180521766
  (0, 15110)	0.1719352837797837
  (1, 366203)	0.24595562404108307
  (1, 348135)	0.4739279595416274
  (1, 256777)	0.28751585696559306
  (1, 217562)	0.40288153995289894
  (1, 145393)	0.575262969264869
  (1, 15110)	0.211037449588008
  (1, 6463)	0.30733520460524466
  (2, 400621)	0.4317732461913093
  (2, 256834)	0.2564939661498776
  (2, 183312)	0.5892069252021465
  (2, 89448)	0.36340369428387626
  (2, 34401)	0.37916255084357414
  :	:
  (319994, 123278)	0.4530341382559843
  (319995, 444934)	0.3211092817599261
  (319995, 420984)	0.22631428606830145
  (319995, 416257)	0.23816465111736276
  (319995, 3

# Training the Machine learning model

Logistic Regression

In [35]:
# Create the model
model=LogisticRegression(max_iter=1000)
model.fit(x_train,y_train)

LogisticRegression(max_iter=1000)

Model Evaluation

Accuracy score

In [36]:
# accuracy score on the training data
x_train_prediction=model.predict(x_train)
training_accuracy=accuracy_score(y_train,x_train_prediction)

In [38]:
print('Accuracy score on the training data:',training_accuracy)

Accuracy score on the training data: 0.81018984375


In [39]:
# accuracy score on the test data
x_test_prediction=model.predict(x_test)
test_accuracy=accuracy_score(y_test,x_test_prediction)


In [40]:
print('Accuracy score on the test data',test_accuracy)

Accuracy score on the test data 0.7780375


Model accuracy =77.8%


Saving the trained model

In [42]:
import pickle

In [43]:
filename="trained_model.sav"
pickle.dump(model,open(filename,'wb'))

Using the saved model for future prediction

In [44]:
# loading the saved model
loaded_model=pickle.load(open('/content/trained_model.sav','rb'))

In [49]:
x_new=x_test[200]
print(y_test[200])

prediction=model.predict(x_new)
print(prediction)

if (prediction[0]==0):
  print('Negative Tweet')
else:
  print('positive Tweet')


1
[1]
positive Tweet


In [54]:
X_new=x_test[3]
print(y_test[3])

prediction=model.predict(X_new)
print(prediction)

if (prediction[0]==0):
  print("Negative Tweet")
else:
  print('Positive Tweet')

0
[0]
Negative Tweet


In [55]:
X_new=x_test[40]
print(y_test[40])

prediction=model.predict(X_new)
print(prediction)

if (prediction[0]==0):
  print("Negative Tweet")
else:
  print('Positive Tweet')

0
[0]
Negative Tweet


In [60]:
X_new=x_test[15]
print(y_test[15])

prediction=model.predict(X_new)
print(prediction)

if (prediction[0]==0):
  print("Negative Tweet")
else:
  print('Positive Tweet')

1
[1]
Positive Tweet
